In [1]:
# 깃허브에서 데이터셋 다운로드하기
!git clone https://github.com/ndb796/Scene-Classification-Dataset-Split 
# 폴더 안으로 이동
%cd Scene-Classification-Dataset-Split

Cloning into 'Scene-Classification-Dataset-Split'...
remote: Enumerating objects: 17038, done.
remote: Total 17038 (delta 0), reused 0 (delta 0), pack-reused 17038
Receiving objects: 100% (17038/17038), 136.02 MiB | 36.58 MiB/s, done.
Checking out files: 100% (17035/17035), done.
/content/Scene-Classification-Dataset-Split


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomCrop((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compos([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder('/train', train_transform)
test_dataset = datasets.ImageFolder('/val', test_transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True, num_workers=2)

In [ ]:
def run(model, criterion, optimizer, data_loader, train=True):
  if train:
    model.train()
  else:
    model.eval()
  
  correct = 0
  total = 0
  loss_sum = 0

  for image, label in data_loader:
    # image = image.to(device)
    # label = label.to(device)

    output = model(image)
    loss = criterion(output, label)
    if train:
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    else:
      _, predict = output.max(1)
      correct += predict.eq(label).sum().item()
      total += label.size(0)
      loss_sum += loss.item()
  
  if not train:
    return correct / total, loss_sum / total

In [2]:
learning_rate = 0.001
momentum = 0.9
num_epoch = 30
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 6)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_data = {'accuracy':[], 'loss':[]}
test_data = {'accuracy':[], 'loss':[]}
for epoch in range(1, num_epoch + 1):
  print(f'Epoch {epoch}')
  run(model, criterion, optimizer, train_dataloader)

  accuracy, loss = run(model, criterion, optimizer, train_dataloader, train=False)
  train_data['accuracy'].append(accuracy)
  train_data['loss'].append(loss)
  print(f'  Train : Accuracy {accuracy}, Loss {loss}')

  accuracy, loss = run(model, criterion, optimizer, test_dataloader, train=False)
  test_data['accuracy'].append(accuracy)
  test_data['loss'].append(loss)
  print(f'  Test : Accuracy {accuracy}, Loss {loss}')

README.md  train/  val/
